---
title: "Multinomial Logit Model"
author: "Vidhi Vashishth"
date: today
---


This assignment expores two methods for estimating the MNL model: (1) via Maximum Likelihood, and (2) via a Bayesian approach using a Metropolis-Hastings MCMC algorithm. 


## 1. Likelihood for the Multi-nomial Logit (MNL) Model

Suppose we have $i=1,\ldots,n$ consumers who each select exactly one product $j$ from a set of $J$ products. The outcome variable is the identity of the product chosen $y_i \in \{1, \ldots, J\}$ or equivalently a vector of $J-1$ zeros and $1$ one, where the $1$ indicates the selected product. For example, if the third product was chosen out of 3 products, then either $y=3$ or $y=(0,0,1)$ depending on how we want to represent it. Suppose also that we have a vector of data on each product $x_j$ (eg, brand, price, etc.). 

We model the consumer's decision as the selection of the product that provides the most utility, and we'll specify the utility function as a linear function of the product characteristics:

$$ U_{ij} = x_j'\beta + \epsilon_{ij} $$

where $\epsilon_{ij}$ is an i.i.d. extreme value error term. 

The choice of the i.i.d. extreme value error term leads to a closed-form expression for the probability that consumer $i$ chooses product $j$:

$$ \mathbb{P}_i(j) = \frac{e^{x_j'\beta}}{\sum_{k=1}^Je^{x_k'\beta}} $$

For example, if there are 3 products, the probability that consumer $i$ chooses product 3 is:

$$ \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{e^{x_1'\beta} + e^{x_2'\beta} + e^{x_3'\beta}} $$

A clever way to write the individual likelihood function for consumer $i$ is the product of the $J$ probabilities, each raised to the power of an indicator variable ($\delta_{ij}$) that indicates the chosen product:

$$ L_i(\beta) = \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} = \mathbb{P}_i(1)^{\delta_{i1}} \times \ldots \times \mathbb{P}_i(J)^{\delta_{iJ}}$$

Notice that if the consumer selected product $j=3$, then $\delta_{i3}=1$ while $\delta_{i1}=\delta_{i2}=0$ and the likelihood is:

$$ L_i(\beta) = \mathbb{P}_i(1)^0 \times \mathbb{P}_i(2)^0 \times \mathbb{P}_i(3)^1 = \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{\sum_{k=1}^3e^{x_k'\beta}} $$

The joint likelihood (across all consumers) is the product of the $n$ individual likelihoods:

$$ L_n(\beta) = \prod_{i=1}^n L_i(\beta) = \prod_{i=1}^n \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} $$

And the joint log-likelihood function is:

$$ \ell_n(\beta) = \sum_{i=1}^n \sum_{j=1}^J \delta_{ij} \log(\mathbb{P}_i(j)) $$



## 2. Simulate Conjoint Data

We will simulate data from a conjoint experiment about video content streaming services. We elect to simulate 100 respondents, each completing 10 choice tasks, where they choose from three alternatives per task. For simplicity, there is not a "no choice" option; each simulated respondent must select one of the 3 alternatives. 

Each alternative is a hypothetical streaming offer consistent of three attributes: (1) brand is either Netflix, Amazon Prime, or Hulu; (2) ads can either be part of the experience, or it can be ad-free, and (3) price per month ranges from \$4 to \$32 in increments of \$4.

The part-worths (ie, preference weights or beta parameters) for the attribute levels will be 1.0 for Netflix, 0.5 for Amazon Prime (with 0 for Hulu as the reference brand); -0.8 for included adverstisements (0 for ad-free); and -0.1*price so that utility to consumer $i$ for hypothethical streaming service $j$ is 

$$
u_{ij} = (1 \times Netflix_j) + (0.5 \times Prime_j) + (-0.8*Ads_j) - 0.1\times Price_j + \varepsilon_{ij}
$$

where the variables are binary indicators and $\varepsilon$ is Type 1 Extreme Value (ie, Gumble) distributed.

The following code provides the simulation of the conjoint data.

(I deleted the code and loaded the code in Python.)



## 3. Preparing the Data for Estimation

The "hard part" of the MNL likelihood function is organizing the data, as we need to keep track of 3 dimensions (consumer $i$, covariate $k$, and product $j$) instead of the typical 2 dimensions for cross-sectional regression models (consumer $i$ and covariate $k$). The fact that each task for each respondent has the same number of alternatives (3) helps.  In addition, we need to convert the categorical variables for brand and ads into binary variables.

In [ ]:
#| label: data-prep-python
#| echo: true

import pandas as pd
import numpy as np

# Examine the structure of the conjoint data
conjoint_data = pd.read_csv("conjoint_data.csv")
print(conjoint_data.head())

# Create dummy variables for brands (using Hulu as reference level)
conjoint_data['netflix'] = np.where(conjoint_data['brand'] == 'N', 1, 0)
conjoint_data['prime'] = np.where(conjoint_data['brand'] == 'P', 1, 0)

# Create dummy variable for ads (No ads is reference)
conjoint_data['ads'] = np.where(conjoint_data['ad'] == 'Yes', 1, 0)

# Organize data in format suitable for MNL estimation
# Number of respondents, tasks, and alternatives per task
n_resp = len(conjoint_data['resp'].unique())
n_tasks = len(conjoint_data['task'].unique())
n_obs = n_resp * n_tasks
n_alts = 3

# Initialize arrays to store the data
choices = np.zeros((n_obs, n_alts))
X_netflix = np.zeros((n_obs, n_alts))
X_prime = np.zeros((n_obs, n_alts))
X_ads = np.zeros((n_obs, n_alts))
X_price = np.zeros((n_obs, n_alts))

# Populate the arrays
row_idx = 0
for r in conjoint_data['resp'].unique():
    for t in range(1, n_tasks + 1):
        row_idx += 1
        
        # Get the data for this respondent and task
        task_data = conjoint_data[(conjoint_data['resp'] == r) & (conjoint_data['task'] == t)]
        
        if len(task_data) == n_alts:
            # Record the choice
            choices[row_idx-1, :] = task_data['choice'].values
            
            # Record the covariates
            X_netflix[row_idx-1, :] = task_data['netflix'].values
            X_prime[row_idx-1, :] = task_data['prime'].values
            X_ads[row_idx-1, :] = task_data['ads'].values
            X_price[row_idx-1, :] = task_data['price'].values

# Combine into a dictionary for easier access
mnl_data = {
    'choices': choices,
    'X_netflix': X_netflix,
    'X_prime': X_prime,
    'X_ads': X_ads,
    'X_price': X_price,
    'n_resp': n_resp,
    'n_tasks': n_tasks,
    'n_alts': n_alts,
    'n_obs': n_obs
}

# Verify we have one choice per task
print(f"Proportion of valid choices: {np.sum(np.sum(choices, axis=1) == 1) / choices.shape[0]}")

The data has now been reshaped into a format suitable for MNL estimation. We've created binary indicator variables for Netflix, Prime (with Hulu as the reference brand), and ads (with ad-free as the reference). For each respondent-task combination, we have a row in our matrix format, with columns representing the different alternatives.


## 4. Estimation via Maximum Likelihood

First, I'll implement the log-likelihood function for the MNL model:

In [ ]:
#| label: mnl-loglik-python
#| echo: true

import numpy as np
from scipy.optimize import minimize
from scipy import linalg
import pandas as pd

# Define the log-likelihood function for the MNL model
def mnl_loglik(beta, data):
    # Extract parameters
    b_netflix = beta[0]
    b_prime = beta[1]
    b_ads = beta[2]
    b_price = beta[3]
    
    # Extract data components
    choices = data['choices']
    X_netflix = data['X_netflix']
    X_prime = data['X_prime']
    X_ads = data['X_ads']
    X_price = data['X_price']
    n_obs = data['n_obs']
    n_alts = data['n_alts']
    
    # Initialize log-likelihood
    loglik = 0
    
    # Loop through each choice task
    for i in range(n_obs):
        # Calculate deterministic utility for each alternative
        V = b_netflix * X_netflix[i, :] + \
            b_prime * X_prime[i, :] + \
            b_ads * X_ads[i, :] + \
            b_price * X_price[i, :]
        
        # Calculate choice probabilities
        exp_V = np.exp(V)
        probs = exp_V / np.sum(exp_V)
        
        # Find the chosen alternative and add its log probability to the log-likelihood
        chosen = np.where(choices[i, :] == 1)[0]
        if len(chosen) > 0:  # Ensure there is a chosen alternative
            loglik += np.log(probs[chosen[0]])
    
    return loglik

# Define the negative log-likelihood for optimization
def neg_mnl_loglik(beta, data):
    return -mnl_loglik(beta, data)

Now, I'll use optim() to find the maximum likelihood estimates:

In [ ]:
#| label: mle-estimation-python
#| echo: true

# Initial parameter values
beta_init = np.array([0, 0, 0, 0])

# Run the optimization to find MLEs
mle_results = minimize(neg_mnl_loglik, beta_init, args=(mnl_data,), 
                      method='BFGS', options={'disp': True})

# Extract the parameter estimates
beta_mle = mle_results.x
param_names = ["Netflix", "Prime", "Ads", "Price"]

# Calculate the Hessian at the optimum using numerical approximation
def compute_hessian(f, x, eps=1e-5, *args):
    n = len(x)
    hessian = np.zeros((n, n))
    fx = f(x, *args)
    
    for i in range(n):
        x_i = x.copy()
        x_i[i] += eps
        fi = f(x_i, *args)
        
        for j in range(n):
            x_j = x.copy()
            x_j[j] += eps
            fj = f(x_j, *args)
            
            x_ij = x.copy()
            x_ij[i] += eps
            x_ij[j] += eps
            fij = f(x_ij, *args)
            
            hessian[i, j] = (fij - fi - fj + fx) / (eps * eps)
    
    return hessian

# Compute the Hessian at the optimum
hessian = compute_hessian(neg_mnl_loglik, beta_mle, 1e-5, mnl_data)

# Calculate standard errors from the Hessian matrix
se_mle = np.sqrt(np.diag(linalg.inv(hessian)))

# Calculate 95% confidence intervals
ci_lower = beta_mle - 1.96 * se_mle
ci_upper = beta_mle + 1.96 * se_mle

In [ ]:
#| label: mle-results-table-python
#| echo: true
#| tbl-cap: Maximum Likelihood Estimates

# Organize results into a DataFrame
mle_table = pd.DataFrame({
    'Parameter': param_names,
    'Estimate': np.round(beta_mle, 3),
    'Std_Error': np.round(se_mle, 3),
    'CI_Lower': np.round(ci_lower, 3),
    'CI_Upper': np.round(ci_upper, 3)
})

# Display the MLE results
mle_table

The table presents our maximum likelihood estimates for the four parameters in the MNL model. 
The estimates are close to the true values used in the simulation: 
$\beta_\text{netflix} = 1.0$, $\beta_\text{prime} = 0.5$, $\beta_\text{ads} = -0.8$, and $\beta_\text{price} = -0.1$. The narrow confidence intervals indicate that our estimates are precise, which is expected given the well-structured simulated data.



## 5. Estimation via Bayesian Methods

Here I'll implement a Metropolis-Hastings MCMC sampler for Bayesian estimation:

In [ ]:
#| label: bayes-log-posterior-python
#| echo: true

from scipy.stats import norm

# Define the log-posterior function (log-likelihood + log-prior)
def log_posterior(beta, data):
    # Calculate log-likelihood
    log_lik = mnl_loglik(beta, data)
    
    # Calculate log-prior
    # N(0,5) priors for the betas on binary variables
    log_prior_netflix = norm.logpdf(beta[0], 0, 5)
    log_prior_prime = norm.logpdf(beta[1], 0, 5)
    log_prior_ads = norm.logpdf(beta[2], 0, 5)
    # N(0,1) prior for the price beta
    log_prior_price = norm.logpdf(beta[3], 0, 1)
    
    # Sum up log-priors
    log_prior = log_prior_netflix + log_prior_prime + log_prior_ads + log_prior_price
    
    # Return log-posterior
    return log_lik + log_prior

In [ ]:
#| label: metropolis-hastings-python
#| echo: true

# Metropolis-Hastings MCMC sampler
def metropolis_hastings(data, n_steps=11000, burnin=1000):
    # Initialize the chain at the MLE estimates for faster convergence
    beta_current = beta_mle.copy()
    
    # Initialize storage for MCMC samples
    beta_samples = np.zeros((n_steps, 4))
    beta_samples[0, :] = beta_current
    
    # Proposal distribution standard deviations
    proposal_sd = np.array([0.05, 0.05, 0.05, 0.005])
    
    # Track acceptance rate
    accepts = 0
    
    # Run the MCMC algorithm
    for s in range(1, n_steps):
        # Generate proposal
        beta_proposal = beta_current + np.random.normal(0, proposal_sd, 4)
        
        # Calculate log acceptance ratio
        log_r = log_posterior(beta_proposal, data) - log_posterior(beta_current, data)
        
        # Accept or reject
        if np.log(np.random.random()) < log_r:
            beta_current = beta_proposal.copy()
            accepts += 1
        
        # Store the current state
        beta_samples[s, :] = beta_current
    
    # Calculate acceptance rate
    acceptance_rate = accepts / (n_steps - 1)
    
    # Return results
    return {
        'samples': beta_samples,
        'post_burnin': beta_samples[burnin:, :],
        'acceptance_rate': acceptance_rate
    }

In [ ]:
#| label: run-mcmc-python
#| echo: true

# Run the MCMC sampler
np.random.seed(456)  # For reproducibility
mcmc_results = metropolis_hastings(mnl_data)

# Report acceptance rate
print(f"MCMC acceptance rate: {mcmc_results['acceptance_rate']:.3f}")

# Extract post-burnin samples
posterior_samples = mcmc_results['post_burnin']

Now, let's examine the trace plot and histogram of the posterior distribution for the Netflix parameter:

In [ ]:
#| label: trace-hist-plots-python
#| echo: true
#| fig-cap: Trace Plot and Posterior Distribution for Netflix Parameter

import matplotlib.pyplot as plt

# Create trace plot and histogram for the Netflix parameter
plt.figure(figsize=(12, 5))

# Trace plot
plt.subplot(1, 2, 1)
plt.plot(posterior_samples[:, 0])
plt.xlabel('Iteration (post-burnin)')
plt.ylabel('Parameter Value')
plt.title('Trace Plot: Netflix Parameter')

# Histogram
plt.subplot(1, 2, 2)
plt.hist(posterior_samples[:, 0], bins=30, color='skyblue', edgecolor='white')
plt.axvline(np.mean(posterior_samples[:, 0]), color='red', linewidth=2)
plt.axvline(np.percentile(posterior_samples[:, 0], 2.5), color='darkred', linewidth=2, linestyle='--')
plt.axvline(np.percentile(posterior_samples[:, 0], 97.5), color='darkred', linewidth=2, linestyle='--')
plt.xlabel('Parameter Value')
plt.title('Posterior Distribution: Netflix Parameter')

plt.tight_layout()
plt.show()

The trace plot shows good mixing of the Markov chain, indicating efficient exploration of the parameter space. The histogram shows the posterior distribution is approximately normally distributed and centered close to the true value of 1.0.

Let's calculate and report the posterior summaries:

In [ ]:
#| label: bayes-summary-stats-python
#| echo: true

# Calculate posterior summary statistics
posterior_means = np.mean(posterior_samples, axis=0)
posterior_sds = np.std(posterior_samples, axis=0)
posterior_ci = np.percentile(posterior_samples, [2.5, 97.5], axis=0).T

In [ ]:
#| label: bayes-results-table-python
#| echo: true
#| tbl-cap: Bayesian Posterior Estimates

# Create a summary table
bayes_table = pd.DataFrame({
    'Parameter': param_names,
    'Mean': np.round(posterior_means, 3),
    'Std_Dev': np.round(posterior_sds, 3),
    'CI_Lower': np.round(posterior_ci[:, 0], 3),
    'CI_Upper': np.round(posterior_ci[:, 1], 3)
})

# Display the Bayesian results
bayes_table

In [ ]:
#| label: comparison-table-python
#| echo: true
#| tbl-cap: 'Comparison: MLE vs Bayesian Estimates'

# Compare with MLE results
comparison_table = pd.DataFrame({
    'Parameter': param_names,
    'MLE': np.round(beta_mle, 3),
    'MLE_CI_Lower': np.round(ci_lower, 3),
    'MLE_CI_Upper': np.round(ci_upper, 3),
    'Bayes_Mean': np.round(posterior_means, 3),
    'Bayes_CI_Lower': np.round(posterior_ci[:, 0], 3),
    'Bayes_CI_Upper': np.round(posterior_ci[:, 1], 3)
})

# Display the comparison
comparison_table

The Bayesian estimates are very similar to the MLE estimates, which is expected given the large sample size and the relatively uninformative priors. Both methods recover the true parameter values quite well, with the posteriors showing slightly wider credible intervals compared to the confidence intervals from MLE.



## 6. Discussion

The parameter estimates from both the MLE and Bayesian approaches align closely with the true values used in the simulation ($\beta_\text{netflix} = 1.0$, $\beta_\text{prime} = 0.5$, $\beta_\text{ads} = -0.8$, $\beta_\text{price} = -0.1$). This confirms that our estimation methods are working correctly.

The interpretation of these parameters provides meaningful insights into consumer preferences:

Brand preferences: The positive and significant coefficients for Netflix (approximately 1.0) and Prime (approximately 0.5) indicate that consumers prefer these brands over Hulu (the reference level). Further, the fact that $\beta_\text{netflix} > \beta_\text{prime}$ means that, all else equal, consumers have a stronger preference for Netflix than for Amazon Prime. In particular, the odds of choosing Netflix over Hulu (if all other attributes are identical) is approximately $e^{1.0} = 2.7$, while the odds of choosing Prime over Hulu is approximately $e^{0.5} = 1.6$.

Ad preference: The negative coefficient for ads (approximately -0.8) indicates that, as expected, consumers dislike advertising in their streaming services. The presence of ads reduces utility and thus decreases the probability of choosing a service with ads, all else equal.

Price sensitivity: The negative coefficient for price (approximately -0.1) reflects that consumers are price-sensitive. As the price increases, the utility decreases, and consequently, the probability of choosing that service decreases. This makes economic sense as consumers typically prefer lower prices.

To extend this model to a multi-level (hierarchical) framework, several key changes would be required:

Parameter heterogeneity: Instead of assuming all respondents have the same preferences ($\beta$s), we would model individual-level parameters that vary across respondents according to a distribution:
$\beta_i \sim \text{MVN}(\mu_\beta, \Sigma_\beta)$

where $\beta_i$ represents the vector of preference parameters for respondent i, $\mu_\beta$ is the vector of population means, and $\Sigma_\beta$ is the covariance matrix capturing heterogeneity across respondents.

Estimation approach: The Bayesian MCMC approach would need to be modified to sample both the individual-level parameters ($\beta_i$) and the population-level hyperparameters ($\mu_\beta$ and $\Sigma_\beta$). This typically involves using a Gibbs sampler with Metropolis-Hastings steps.

Data structure: The data preparation would remain similar, but we would need to keep track of respondent identities more carefully to model the within-respondent correlation in choices.

Computational complexity: The model would become significantly more complex, with hundreds of parameters to estimate (4 parameters per respondent × 100 respondents, plus population parameters), requiring more efficient MCMC algorithms and potentially more computational resources.

Prior specifications: We would need to specify priors not only for the mean parameters but also for the covariance matrix, typically using an Inverse-Wishart distribution.